In [1]:
import sys
sys.path.append('..')
# reload local packages automatically
%load_ext autoreload
%autoreload 2

from jax import config
config.update("jax_enable_x64", True)

# Import packages.
import cvxpy as cp
import numpy as np

import matplotlib.pyplot as plt


from opentn.transformations import create_kitaev_liouvillians, exp_operator_dt, factorize_psd, super2choi
d, N = 2, 4
dim = d**N
Lvec, Lvec_odd, Lvec_even, Lnn = create_kitaev_liouvillians(N=N, d=d, gamma=1e-2)
tau = 4
superops_exp = []
for i, op in enumerate([Lvec, Lvec_odd, Lvec_even]):
    if i == 1:
        superops_exp.append(exp_operator_dt(op, tau/2, 'jax'))
    else:
        superops_exp.append(exp_operator_dt(op, tau, 'jax'))
exp_Lvec, exp_Lvec_odd, exp_Lvec_even = superops_exp

X1 = factorize_psd(psd=super2choi(exp_Lvec_odd), tol=1e-12)
X2 = factorize_psd(psd=super2choi(exp_Lvec_even), tol=1e-12)

from opentn.optimization import gds, frobenius_norm, model_Ys, compute_loss
cost_n4, grads_n4, xi_n4 = gds(fn=compute_loss, x0=[X1, X2, X1], iter=int(2e3), loss_fn=frobenius_norm, model=model_Ys, rate=1.5e-9, exact = exp_Lvec, show_cost=False)
print(cost_n4[0])
print(cost_n4[-1])

2.638300722408311e-07
1.4977787041165522e-07


In [2]:
from opentn.transformations import super2choi, choi_composition
from opentn.optimization import small2zero
from scipy import sparse

# rest of matrices optimized with GSD
X1_gds, X2_gds, X3_gds = xi_n4[-1]
Y1_gds = X1_gds@X1_gds.conj().T
Y2_gds = X2_gds@X2_gds.conj().T
Y3_gds = X3_gds@X3_gds.conj().T

tol = 1e-12

# rhs to which we want to approximate
rhs = np.asarray(super2choi(superop=exp_Lvec, dim=dim))
# srhs = sparse.csr_matrix(rhs) # small2zero was deleted
srhs = sparse.csr_matrix(small2zero(rhs, tol=tol)) # small2zero was deleted
srhs = srhs.astype(np.float64)

/Users/emiliano_gr/opt/anaconda3/envs/cvxpy-env/lib/python3.10/site-packages/scipy/sparse/_data.py:72: ComplexWarning: Casting complex values to real discards the imaginary part
  self.data.astype(dtype, casting=casting, copy=True),


In [3]:
from opentn.transformations import link_product_cvxpy
Y1_cvxpy = cp.Variable((dim**2,dim**2), PSD=True)
C2 = choi_composition(small2zero(Y2_gds, tol), small2zero(Y3_gds, tol))
lhs = link_product_cvxpy(C1=Y1_cvxpy, C2=C2, dim=dim, transpose=1, optimization=True)
cost = cp.norm(lhs - srhs, "fro")
prob = cp.Problem(cp.Minimize(cost))
Y1_cvxpy.value = small2zero(Y1_gds, tol=tol).astype(np.float64) 
cost.value

/var/folders/_n/xw2mchz92t9_fcbxg7lwsdwh0000gn/T/ipykernel_45846/826646750.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  Y1_cvxpy.value = small2zero(Y1_gds, tol=tol).astype(np.float64)


1.4977447039683222e-07

In [4]:
prob.solve(solver=cp.CLARABEL, verbose=True, canon_backend=cp.SCIPY_CANON_BACKEND, tol_feas=tol, tol_gap_abs=tol)

                                     CVXPY                                     
                             v1.4.0.dev0+0.ec21400                             
(CVXPY) Jul 29 09:26:37 PM: Your problem has 65536 variables, 0 constraints, and 0 parameters.
(CVXPY) Jul 29 09:26:37 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 29 09:26:37 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 29 09:26:37 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 29 09:26:37 PM: Compiling problem (target solver=CLARABEL).
(CVXPY) Jul 29 09:26:37 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuf

(CVXPY) Jul 29 09:26:37 PM: Applying reduction Dcp2Cone
(CVXPY) Jul 29 09:26:37 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 29 09:26:37 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Jul 29 09:47:31 PM: Applying reduction CLARABEL
(CVXPY) Jul 29 09:47:31 PM: Finished problem compilation (took 1.254e+03 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jul 29 09:47:31 PM: Invoking solver CLARABEL  to obtain a solution.


: 

: 